In [1]:
# Importing the necessary libraries
import os  # For working with files and directories
import torch  # PyTorch
import torchvision.transforms as transforms  # image transformation
from torchvision.models import resnet50  # Importing the ResNet50 model
from PIL import Image  # image processing library
import psycopg2  # PostgreSQL database adapter

In [2]:
def load_model():
    """
    The pre-trained ResNet50 model was loaded and set to evaluation mode.

    Returns: 
        model: loaded ResNet50 model
    """
    model = resnet50(pretrained=True)  # Loading pre-trained models
    model.eval()  # Setting up the model for evaluation mode
    return model

In [13]:
from torchvision.models import resnet50
import torch.nn as nn

def load_model():
    """
    Load the pre-trained ResNet50 model, 
        remove the last fully connected layer, 
        and set to evaluation mode.
    
    Returns:
        model: Loaded ResNet50 feature extraction model
    """
    model = resnet50(pretrained=True)  # Loading pre-trained models
    # Remove the last fully connected layer
    model = nn.Sequential(*list(model.children())[:-1])  # Getting the output of the feature extraction layer
    model.eval()  # Setting up the model for evaluation mode
    return model

In [3]:
def preprocess_image(image_path):
    """
    Preprocess the image.

    parameters:
        image_path (str): Image File Path

    Returns:
        torch.Tensor: Preprocessed image tensor
    """
    # Defining Image Transformation Operations
    transform = transforms.Compose([
        transforms.Resize(256),  # Resizing images
        transforms.CenterCrop(224),  # Center Cropped Image
        transforms.ToTensor(),  # Converting images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # standardization
    ])
    
    image = Image.open(image_path).convert("RGB")  # Read images and convert to RGB format
    image = transform(image).unsqueeze(0)  # Add Batch Dimension
    return image

In [4]:
def generate_image_vector(model, image_path):
    """
    Generate a 2048-dimensional vector of images.

    parameters:
        model: Pre-trained ResNet50 model
        image_path (str): Image File Path

    Returns:
        numpy.ndarray: 2048-dimensional image vector
    """
    image = preprocess_image(image_path)  # Preprocessed images
    with torch.no_grad():  # Disable gradient calculation
        vector = model(image).numpy()  # Generate vectors using models and convert to numpy arrays
    return vector.flatten()  # Flatten vectors for easy storage

In [5]:
def insert_vector_to_db(connection, image_name, vector):
    """
    Inserts image vectors into a PostgreSQL database.

    parameters:
        connection: PostgreSQL Database Connection
        image_name (str): Image file name (no extension)
        vector (numpy.ndarray): 2048-dimensional image vector
    """
    cursor = connection.cursor()  # Creating Database Cursors
    # Perform an insertion operation
    cursor.execute(
        "INSERT INTO image_info (image_name, image_path, vector) VALUES (%s, %s, %s)",
        (image_name, None, vector.tolist())  # image_path Temporarily empty
    )
    connection.commit()  # Submission of transactions
    cursor.close()  # Close cursor


In [6]:
def process_images_in_directory(image_dir, model, connection):
    """
    Recursively read all the images in the catalog and insert their vectors into the database.

    parameters:
        image_dir (str): Root directory of the image file
        model: Pre-trained ResNet50 model
        connection: PostgreSQLdatabase connection
    """
    for root, dirs, files in os.walk(image_dir):  # Iterate through the catalog
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Processes only images of a specific format
                image_path = os.path.join(root, file)  # Get the full image path
                print(f"Processing {image_path}...")  # Print the path of the image being processed
                vector = generate_image_vector(model, image_path)  # Generate image vectors
                image_name = os.path.splitext(file)[0]  # Use the file name (without extension) as the ID
                insert_vector_to_db(connection, image_name, vector)  # Inserting vectors into the database


In [18]:
def main(image_dir):
    """
    main function that executes the entire process.

    parameters:
        image_dir (str): Root directory of the image file
    """
    # Connecting to a PostgreSQL Database
    connection = psycopg2.connect(
        dbname='imageSearch',  # database name
        user='postgres',  # user ID
        password='test-postgres',  # cryptographic
        host='localhost',  # RDSTerminal node of the instance
        port='5432'  # PostgreSQL ports
    )

    model = load_model()  # Loading Models
    process_images_in_directory(image_dir, model, connection)  # process image
    connection.close()  # Close the database connection


In [19]:
# Execute the main function with the image directory as an argument
main('/Users/Tommy/AI/image-search/image-search-pgvector/image-test') 

Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0249136006.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0244267027.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0244267032.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0240670058.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0241436001.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0249136011.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0241590001.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0240840032.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0244269020.jpg...
Processing /Users/Tommy/AI/image-search/image-search-pgvector/image-test/024/0244853036.jpg...
Processing /Users/Tommy/AI/image-search/image-sear